╔══<i><b>Alai-DeepLearning</b></i>════════════════════════════╗
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 13. RNN-Basis**
# Section 6. LSTM 이해하기



### _Objective_

1. LSTM의 코드를 좀 더 정리하여 가독성을 높여보도록 하겠습니다. <br>
2. LSTM이 왜 RNN보다 우수한지 분석해 보도록 하겠습니다. <br>
3. 다양한 LSTM의 변형 형태를 탐색해 보도록 하겠습니다. <br>

╚═════════════════════════════════════════╝

### Reference List

1. [LSTM과 ResNet](http://tmmse.xyz/2016/10/15/lstm-resnet/)
2. 밑바닥부터 시작하는 딥러닝 2

In [3]:
from tensorflow.keras.layers import Layer
import tensorflow as tf
import tensorflow.keras.backend as K

/Users/ksj/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


<br>

# \[ 1. LSTM 정리하기 \]
---
----
> *현재 구현체는 수식이 복잡합니다. 이를 좀 더 단순하게 정리해보도록 하겠습니다.*<br>


![Imgur](https://imgur.com/4m5fgb9.png)

* 아래는 LSTM에서 수행하는 계산을 정리한 수식들입니다.

$
{gate}^{(f)} = \sigma(x_t\cdot W_x^{(f)}+ h_t\cdot W_h^{(f)}+b^{(f)}) \\
{gate}^{(o)} = \sigma(x_t\cdot W_x^{(o)}+ h_t\cdot W_h^{(o)}+b^{(o)}) \\
{gate}^{(i)} = \sigma(x_t\cdot W_x^{(i)}+ h_t\cdot W_h^{(i)}+b^{(i)}) \\
update = tanh(x_t\cdot W_x^{(u)}+ h_t\cdot W_h^{(u)}+b^{(u)})\\
----------\\
c_t = {gate}^{(f)} \odot c_{t-1} + {gate}^{(i)} \odot update\\
h_t = {gate}^{(o)} \odot tanh(c_t)
$

<br>

## 1. 수식을 단순하게 만들기
---

* 현재 Gate의 연산에서는 총 8번의 행렬곱 연산이 존재합니다.<br>
* 이러한 행렬곱 연산만을 따로 빼내 정리하면 아래와 같습니다.

    $
    x_t\cdot W_x^{(f)}+ h_t\cdot W_h^{(f)}+b^{(f)} \\
    x_t\cdot W_x^{(o)}+ h_t\cdot W_h^{(o)}+b^{(o)} \\
    x_t\cdot W_x^{(i)}+ h_t\cdot W_h^{(i)}+b^{(i)} \\
    x_t\cdot W_x^{(u)}+ h_t\cdot W_h^{(u)}+b^{(u)} \\
    $<br>

* 이 식을 정리하면 아래처럼 구현이 됩니다.<br>
$
x_t \cdot [W_x^{(f)},  W_x^{(o)},  W_x^{(i)},  W_x^{(u)}] + h_t \cdot [W_h^{(f)},  W_h^{(o)},  W_h^{(i)},  W_h^{(u)}] + [b^{(f)}, b^{(o)}, b^{(i)}, b^{(u)}] \\
= x_t \cdot W_x + h_t \cdot W_h + b
$

<br>

## 2. build 구현하기
---

* 수식을 위와 같이 정리하게 되면, 한 번에 8개의 가중치를 하나로 모아 관리할 수 있게 됩니다.<br>
* 이로 인해 `self.build()`내 가중치 선언이 매우 간결해집니다.<br>

In [2]:
class LSTMCell(Layer):
    def __init__(self, n_units, **kwargs):
        self.n_units = n_units
        self.state_size = (n_units, n_units)
        super(LSTMCell, self).__init__(**kwargs)
        
    def build(self, input_shape):
        # Forget Gate에 관련된 weight 선언
        self.wx = self.add_weight("weight_x",
                                  shape=(input_shape[-1],self.n_units*4),
                                  initializer='glorot_uniform')
        self.wh = self.add_weight('weight_h',
                                  shape=(self.n_units,self.n_units*4),
                                  initializer='orthogonal')
        self.b = self.add_weight('bias',
                                 shape=(self.n_units*4,),
                                 initializer='zeros')
        super(LSTMCell, self).build(input_shape)

## 3. call 구현하기
---

* 위와 같이 정리할 경우, 큰 행렬을 한번에 계산할 수 있어 좀 더 빨라집니다. <br>
* 그리고 이후 계산은 slice를 통해 일부 행렬만 가져와 forget gate, input gate, output gate를 계산하게 됩니다.

In [4]:
class LSTMCell(Layer):
    def __init__(self, n_units, **kwargs):
        self.n_units = n_units
        self.state_size = (n_units, n_units)
        super(LSTMCell, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.wx = self.add_weight("weight_x",
                                  shape=(input_shape[-1],self.n_units*4),
                                  initializer='glorot_uniform')
        self.wh = self.add_weight('weight_h',
                                  shape=(self.n_units,self.n_units*4),
                                  initializer='orthogonal')
        self.b = self.add_weight('bias',
                                 shape=(self.n_units*4,),
                                 initializer='zeros')
        super(LSTMCell, self).build(input_shape)
        
    def call(self, x, states):
        h, c = states
        # 행렬곱 연산
        z = tf.dot(x, self.wx) + tf.dot(h, self.wh) + self.b
        
        # forget에 관련된 처리들
        forget_fate = tf.sigmoid(z[:,:self.n_units])
        # update에 관련된 처리들
        input_gate = tf.sigmoid(z[:,self.n_units:self.n_units*2])        
        update = tf.tanh(z[:,self.n_units*2:self.n_units*3])
        new_c = forget_gate * c + update * input_gate
        # output에 관련된 처리들
        output_gate = tf.sigmoid(z[:,self.n_units*3:])
        new_h = output_gate * tf.tanh(new_c)

        return output, [new_h, new_c]    

### (3) Keras에서 제공해주는 LSTMCell과 LSTM

Keras에서는 SimpleRNNCell과 같이, LSTMCell에 대한 Class를 제공해줍니다.<br>
이를 통해 우리는 좀 더 편하게 LSTMCell을 구현할 수 있습니다.<br>

In [14]:
from tensorflow.keras.layers import LSTMCell, RNN, Input, Dense
from tensorflow.keras.models import Model

K.clear_session()
n_steps = 30
n_inputs = 50
n_hiddens = 100
n_outputs = 3

inputs = Input(shape=(n_steps,n_inputs))
hidden = RNN(LSTMCell(n_hiddens))(inputs)
output = Dense(n_outputs,activation='softmax')(hidden)

model = Model(inputs,output)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 30, 50)            0         
_________________________________________________________________
rnn (RNN)                    (None, 100)               60400     
_________________________________________________________________
dense (Dense)                (None, 3)                 303       
Total params: 60,703
Trainable params: 60,703
Non-trainable params: 0
_________________________________________________________________


그리고 좀 더 간결하게, RNN과 LSTMCell을 한번에 호출하고자 하면, LSTM을 이용하면 됩니다.

In [13]:
from tensorflow.keras.layers import LSTM

K.clear_session()
n_steps = 30
n_inputs = 50
n_hiddens = 100
n_outputs = 3

inputs = Input(shape=(n_steps,n_inputs))
hidden = LSTM(n_hiddens)(inputs)
output = Dense(n_outputs,activation='softmax')(hidden)

model = Model(inputs,output)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 30, 50)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               60400     
_________________________________________________________________
dense (Dense)                (None, 3)                 303       
Total params: 60,703
Trainable params: 60,703
Non-trainable params: 0
_________________________________________________________________


<br>

# \[ 2. LSTM 분석하기 \]
---
---

> *LSTM은 Long Short-Term Memory를 줄인 이름입니다. 이 용어 그래도 LSTM의 주된 특징은 단기 기억(short-term memory)를 긴(Long) 시간동안 지속할 수 있다는 점에 있습니다.*<br>



<br>

## 1. LSTM의 순전파

---

* LSTM은 상태 정보와 기억 정보를 나눔으로써, 꼭 필요한 장기 정보를 보존할 수 있도록 설계되어 있습니다.<br>
* 아래의 그림과 같이 기억 셀에는 원소 간 곱만이 존재할 뿐, 별다른 Weight와의 행렬곱이 없습니다. 이는 지켜야할 정보는 유실되지 않은 채로 쭉 갈 수 있음을 의미합니다.

![Imgur](https://imgur.com/lVWxQeW.png)

<br> 

## 2. LSTM의 역전파
---


* 기억 셀의 역전파에서는 더하기와 원소 간 곱하기로만 구현되어 있습니다. 더하기 연산자에서는 역전파를 통해 오는 기울기를 그대로 흘려 줍니다.<br>
* 원소 간 곱하기 연산자는 RNN에서의 행렬곱과 달리, 매번 다른 원소별 곱이 이루어지기 때문에, Gradient Vanishing이 쉽게 발생하지 않습니다.<Br>
* 또한 모든 Gate의 출력값은 0~1 사이로 정해져 있기 때문에, Gradient Exploding을 막아줍니다.
    
![Imgur](https://imgur.com/KloK4Yg.png)

* 위와 같은 Gradient의 흐름은 이전에 배웠던 ResNet과 매우 유사합니다.<br>
* ResNet은 Identity Function을 통해, Gradient을 전파함으로써, 매우 깊은 모델에서도 효과적으로 Gradient을 전달할 수 있게 됩니다.

![Imgur](https://imgur.com/1GKuhha.png)

<br>

## 3. LSTM의 메모리와 연산량
----


* LSTM은 RNN에 비해 훨씬 더 많은 Weight를 가지고 있습니다. 3개의 gate와 하나의 update로 인해, 총 4배의 weight가 필요합니다.<br>
* 또한 추가적으로 3번의 sigmoid 연산과 1번의 tanh 연산이 더해지기 때문에, Cell에서 처리하는 데에 좀 더 많은 시간을 필요로 합니다.<br>



<br>

# 3. LSTM의 다양한 변형들
---
---


## 1. PeepHole LSTM

기존의 LSTM에서는 GATE를 제어할 때, 입력($x_t$)와 이전 상태($h_{(t-1)}$으로만 조절하였습니다. <br> 대신 게이트 제어기에서 장기 상태도 조금 노출시켜 좀 더 많은 문맥을 감지할 수 있도록 만든 것이 아래의 PeepHole LSTM 입니다.

![Imgur](https://i.imgur.com/xoYH7On.png)

* 이들이 제안한 것은 핍홀 연결(peephole connection)이라 부르는 추가적인 연결이 있는 LSTM 변종입니다. 이전 기억 셀의 정보를 Gate의 input값으로 넣습니다. 좀 더 성능이 낫지만, 추가적인 연산량이 필요로 하다는 단점이 있습니다.

## 2. GRU Cell

2014년에 조경현 교수님께서 제안한 GRU(Gated Recurrent Unit)은 연산이 많고 복잡한 LSTM을 좀 더 단순하게 만든 형태입니다.

![Imgur](https://i.imgur.com/U7GqrAw.png)

* 기억과 상태 벡터를 하나의 벡터로 다시 합쳤습니다.
* 하나의 게이트($z_t$)가 삭제와 입력 모두를 제어합니다. 게이트($z_t$)의 값이 1이면 삭제 게이트가 열리고 입력 게이트가 닫힙니다. 게이트의 값이 0이면 삭제 게이트가 닫히고 입력 게이트가 열립니다.

#  

---

    Copyright(c) 2019 by Public AI. All rights reserved.<br>
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2019/06/18

---